In [ ]:
import time, json, serial, math, ssl, time, queue, socket, struct, random, threading, adafruit_motorkit
 
def joystick(a0,a1): 
    x, y = (2048 - a1)/2049, (2048 - a0)/2049  
    r = math.sqrt(x*x+y*y) 
    if r > 1: r = 1 
    left, right = r, r
    theta = math.atan2(y, x) / math.pi 
    if   theta >  0.5: # II       1  ~  0.5    -1 ~  1
        left  *=   -4*theta + 3   
    elif theta >    0: # I      0.5  ~  0       1 ~ -1 
        right *=    4*theta - 1   
    elif theta > -0.5: #  IV      0  ~ -0.5     0 ~ -1 
        left  *=   -1    
        right *=    2*theta  
    else:              # III    -0.5 ~ -1      -1 ~  0
        left  *=   -2*theta - 2 
        right *=   -1   
    return left, right 
 
kit = adafruit_motorkit.MotorKit() 
m1, m2, m3, m4 = kit.motor1 , kit.motor2 , kit.motor3 , kit.motor4 

def motor(a0, a1):
    left, right = joystick(a0, a1) 
    if abs(left) < 0.3: left = 0
    if abs(right) < 0.3: right = 0
    m1.throttle, m2.throttle = right, right
    m3.throttle, m4.throttle =  left,  left
    
motor(4096, 2048)
time.sleep(0.5)
motor(2048, 2048)

yaw_pin      = 19
pitch_pin    = 26
gpio_port    = 8889 
control_port = 54322  
sensor_port  = 54324
server_ip    = 'xg590.xyz' 
retry_in     = 30 # seconds

import subprocess, socket, pigpio
def startGPIOdaemon():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)                                            #2 Second Timeout
    result = sock.connect_ex(('127.0.0.1', gpio_port))
    if result == 0:
        print( 'pigpiod is running' )
    else:
        subprocess.run("sudo pkill pigpiod", shell=True)
        subprocess.run("sudo pigpiod -p 8889", shell=True) 
        print( 'pigpiod was just started' ) 
    sock.close()
startGPIOdaemon()

import pigpio, time, adafruit_motorkit
pi  = pigpio.pi(port=gpio_port) ### Start PiGPIO Client http://abyz.me.uk/rpi/pigpio/pigpiod.html 


def servo(a2, a3): 
    a2 = a2 * 0.2441 + 1500 
    a3 = a3 * 0.4516 + 650 
    pi.set_servo_pulsewidth(pitch_pin, a2) #1500 2500
    pi.set_servo_pulsewidth(  yaw_pin, a3)   # 650 2500
servo(0, 0)
time.sleep(2)
servo(4096, 4096)

In [ ]:
# print(joystick(   0 , 1840)) 
# print(joystick(4096 , 1840)) 
# print(joystick(1840 , 4096)) 
# print(joystick(1840 ,    0)) 
#joystick(-1,0)
#joystick(-0.7,0.7)
#print('____________')
#joystick(0,1)
#joystick(0.7,0.7)
#print('____________')
#joystick(1,0)
#joystick(0.7,-0.7)
#print('____________')
#joystick(0,-1)
#joystick(-0.7,-0.7)  
#up       0 1840       0        1840 
#down  4096 1840       0       -1840 
#left  1840 4096   -1840           0 
#right 1840    0    1840           0  

In [ ]:
# Establish connection with server
import ssl, struct, threading   
server_cert = 'setup/secret/server.crt'
client_cert = 'setup/secret/rover.crt'
client_key  = 'setup/secret/rover.key'  

context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert)
context.load_cert_chain(certfile=client_cert, keyfile=client_key) 

def control_link_daemon(): 
    while True: 
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
        ss.settimeout(retry_in)  
        try: 
            print('[Control Link] Try to connect the server') 
            ss.connect((server_ip, control_port))  
            print('[Control Link] Succeed a connection')  
        except (socket.timeout, ConnectionRefusedError, ConnectionResetError):
            print(f'[Control Link] Failed to connect the server and retry the connection in {retry_in} seconds')
            time.sleep(retry_in) 
            continue  
        fmt = '<HHHHBBB'
        size = struct.calcsize(fmt)
        while True:
            try:
                a0, a1, a2, a3, d0, d1, d2 = struct.unpack(fmt, ss.recv(size))  
                #motor(a0, a1)
                servo(a2, a3)
                
            except Exception as e:
                ss.shutdown(socket.SHUT_RDWR) 
                ss.close()
                print(f'[Control Link] Connection closed and will reopen in {retry_in} seconds', e)
                time.sleep(retry_in)
                break 
                
control_link_thread = threading.Thread(target=control_link_daemon); control_link_thread.start(); print('control_link_daemon', control_link_thread.getName())

In [ ]:
def sensor_link_daemon():
    sensor = serial.Serial(port='/dev/ttyS0', baudrate=115200, timeout=3) # serial commu with GPS chip  
    while True:   
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
        ss.settimeout(retry_in)  
        try: 
            print('[Sensor  Link] Trying to connect the server')
            ss.connect((server_ip, sensor_port))
            print('[Sensor  Link] Succeed a connection')  
        except (ConnectionRefusedError, ConnectionResetError, socket.timeout):
            print(f'[Sensor  Link] Failed to connect the server and retry the connection in {retry_in} seconds')
            time.sleep(retry_in)
            continue
        try:
            while True: 
                data = sensor.readline()
                js = json.loads(data)
                if 'Sat' in js:  
                    ss.send(struct.pack('<H', len(data))) 
                    ss.send(data)
        except socket.timeout as e: 
            ss.shutdown(socket.SHUT_RDWR)
            ss.close()
            print(f'[Sensor  Link] Time out and closed. Reestablish in {retry_in} seconds:', e)
            time.sleep(retry_in) 
        except (ConnectionResetError, BrokenPipeError) as e:
            print(f'[Sensor  Link] Server ended the connection. Reestablish in {retry_in} seconds:', e)
            time.sleep(retry_in) 

sensor_link_thread = threading.Thread(target=sensor_link_daemon); sensor_link_thread.start(); print('sensor_link_daemon', sensor_link_thread.getName())